In [2]:
import os
import sys
from music21 import converter, note, chord, key, harmony
import sys

sys.path.append("./../../model")
sys.path.append("./../../model/src/tokenizer")
os.environ["RWKV_JIT_ON"] = "1"
os.environ["RWKV_CUDA_ON"] = "0"

In [3]:
def analyze_melodic_similarity(score, reference_score):
    generated_melody = score.flat.notes
    reference_melody = reference_score.flat.notes
    generated_pitches = [
        n.pitch.midi for n in generated_melody if isinstance(n, note.Note)]
    reference_pitches = [
        n.pitch.midi for n in reference_melody if isinstance(n, note.Note)]
    matching_pitches = sum(1 for gp, rp in zip(
        generated_pitches, reference_pitches) if gp == rp)
    similarity_ratio = matching_pitches / \
        min(len(generated_pitches), len(reference_pitches))
    return similarity_ratio

In [4]:
def analyze_rhythm_consistency(score):
    notes = score.flat.notes
    durations = [n.quarterLength for n in notes if isinstance(n, note.Note)]
    mean_duration = sum(durations) / len(durations)
    variance = sum((d - mean_duration) **
                   2 for d in durations) / len(durations)
    std_dev = variance ** 0.5
    return std_dev

In [5]:
def analyze_harmonic_coherence(score):
    key_analysis = score.analyze('key')

    chords = score.chordify()
    chord_list = [ch for ch in chords.recurse().getElementsByClass('Chord')]

    consonance_count = sum(1 for ch in chord_list if ch.isConsonant())
    dissonance_count = len(chord_list) - consonance_count
    total_chords = len(chord_list)
    consonance_ratio = consonance_count / total_chords
    dissonance_ratio = dissonance_count / total_chords

    return consonance_ratio, dissonance_ratio

In [6]:
def analyze_tonal_stability(score):
    key_changes = score.flat.getElementsByClass(key.Key)
    num_key_changes = len(key_changes)
    return num_key_changes

In [8]:

import logging
import glob
import uuid
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
sys.path.append("./../../model/src/tokenizer")
print(sys.path)
from rwkv.utils import PIPELINE, PIPELINE_ARGS
from rwkv.model import RWKV
from src.tokenizer.midi_to_str import convert_midi_bytes_to_str
from src.tokenizer.midi_util import convert_str_to_midi

model = RWKV(model='./../../../rwkv-pinao.pth', strategy="cpu fp32")
pipeline = PIPELINE(
    model, './../../model/src/tokenizer/tokenizer-midi/tokenizer.json')
args = PIPELINE_ARGS(
    temperature=1.0,
    top_p=0.8,
    top_k=8,
    # alpha_frequency=0.25,
    # alpha_presence=0.25,
    alpha_decay=0.997,  # gradually decay the penalty
    token_stop=["<end>"],
    chunk_len=512,
)

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


def analyze_midi_file(filename):
    """
    Analyze a single MIDI file and calculate metrics.
    """
    output_filename = f"mdiOut_{uuid.uuid4()}.mid"
    logger.info(f"Processing file: {filename}")
    with open(os.path.join('./validation', filename), 'rb') as f:
        data = []
        pipeline.generate(
            convert_midi_bytes_to_str(None, ('', f.read()))[1][0], args=args, callback=lambda token: data.append(token))
        convert_str_to_midi(" ".join(map(str, data)), 300, 0).save(
            output_filename
        )
    reference_score = converter.parse(os.path.join('./validation', filename))
    score = converter.parse(output_filename)
    logger.info(f"Finished processing file: {filename}")
    return {
        'rhythm_consistency': analyze_rhythm_consistency(score),
        'consonance_ratio': analyze_harmonic_coherence(score)[0],
        'dissonance_ratio': analyze_harmonic_coherence(score)[1],
        'melodic_similarity': analyze_melodic_similarity(score, reference_score),
        'tonal_stability': analyze_tonal_stability(score)
    }


def calculate_metrics():
    """
    Calculate overall metrics for all MIDI files in the validation directory.
    """
    total = 0
    metrics = {'rhythm_consistency': 0, 'consonance_ratio': 0,
               'dissonance_ratio': 0, 'melodic_similarity': 0, 'tonal_stability': 0}
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for filename in os.listdir('./validation')[:100]:
            if filename.endswith(".mid") or filename.endswith(".midi"):
                futures.append(executor.submit(analyze_midi_file, filename))
        for i, future in enumerate(futures):
            result = future.result()
            total += 1
            for metric, value in result.items():
                metrics[metric] += value
            logger.info(f"Processed {total} files")
            if i % 10 == 0:
                logger.info(
                    f"  {metric.capitalize().replace('_', ' ')}: {value / total:.2f}")
    logger.info(
        f"  {metric.capitalize().replace('_', ' ')}: {value / total:.2f}")
    for metric, value in metrics.items():
        logger.info(
            f"  {metric.capitalize().replace('_', ' ')}: {value / total:.2f}")

calculate_metrics()

['/home/arthur/miniconda3/envs/AItuneCraft/lib/python310.zip', '/home/arthur/miniconda3/envs/AItuneCraft/lib/python3.10', '/home/arthur/miniconda3/envs/AItuneCraft/lib/python3.10/lib-dynload', '', '/home/arthur/.local/lib/python3.10/site-packages', '/home/arthur/miniconda3/envs/AItuneCraft/lib/python3.10/site-packages', './../../model', './../../model/src/tokenizer', './../../model/src/tokenizer', './../../model/src/tokenizer']
RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ./../../../rwkv-pinao.pth ...
Model detected: v6.0
Strategy: (total 20+1=21 layers)
* cpu [float32, float32], store 21 layers
0-cpu-float32-float32 1-cpu-float32-float32 2-cpu-float32-float32 3-cpu-float32-float32 4-cpu-float32-float32 5-cpu-float32-float32 6-cpu-float32-float32 7-cpu-float32-float32 8-cpu-float32-float32 9-cpu-float32-float32 10-cpu-float32-float32 11-cpu-float32-float32 12-cpu-float32-float32 13-cpu-float32-float32 14-cpu-float32-float32 15-cpu-float32-float32 16-cpu-float32-float32 17-cpu-

2024-09-18 00:53:33,169 - INFO - Processing file: 1.mid
2024-09-18 00:53:33,171 - INFO - Processing file: 10.mid
2024-09-18 00:53:33,172 - INFO - Processing file: 100.mid
2024-09-18 00:53:33,172 - INFO - Processing file: 1000.mid
2024-09-18 00:53:33,173 - INFO - Processing file: 1001.mid
2024-09-18 00:53:47,461 - INFO - Finished processing file: 1001.mid
2024-09-18 00:53:47,672 - INFO - Processing file: 1002.mid
2024-09-18 00:53:47,909 - INFO - Finished processing file: 1.mid
2024-09-18 00:53:48,150 - INFO - Processing file: 1003.mid
2024-09-18 00:53:48,150 - INFO - Processed 1 files
2024-09-18 00:53:48,153 - INFO -   Tonal stability: 0.00
2024-09-18 00:53:49,044 - INFO - Finished processing file: 100.mid
2024-09-18 00:53:49,403 - INFO - Processing file: 1004.mid
2024-09-18 00:53:51,426 - INFO - Finished processing file: 1000.mid
2024-09-18 00:53:51,670 - INFO - Processing file: 1005.mid
2024-09-18 00:53:59,115 - INFO - Finished processing file: 10.mid
2024-09-18 00:53:59,404 - INFO - 